In [ ]:
from block_ai.lib.myblokus import point
from block_ai.lib.myblokus.point import Point
from block_ai.lib.myblokus.piece import Piece
from block_ai.lib.myblokus.corner import Corner
from block_ai.lib.myblokus.orientation import Orientation

import itertools
import numpy as np
import random

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [ ]:
class Move:
    
    def __init__(self, orientation, player_id, piece_id, corner):
        self.orientation = orientation
        self.player_id = player_id
        self.piece_id = piece_id
        self.corner = corner

    def get_footprint(self):
        return [p for p in self.orientation.points]
    
    def __repr__(self):
        o_repr = repr(self.orientation)
        c_repr = repr(self.corner)
        return f"""Move(orientation={o_repr},
                        player_id={self.player_id},
                        piece_id='{self.piece_id}',
                        corner={c_repr})"""
    
    def __eq__(self, other):
        for attr in ["orientation", "player_id", "piece_id", "corner"]:
            if not eval(f"self.{attr} == other.{attr}"):
                return False
        return True

    def __str__(self):
        return f"Player ID: {self.player_id}\nPiece ID:{self.piece_id}\n{self.orientation}"

    
class Board:
    SIDE_LENGTH = 20
    EMPTY = -1
    
    def __init__(self):
        shape = (self.SIDE_LENGTH, self.SIDE_LENGTH)
        self.board = np.full(shape, self.EMPTY)
        
    def update(self, move):
        for p in move.get_footprint():
            self.assign(p, move.player_id)
        
    def are_squares_free(self, orientation):
        for p in orientation.points:
            if not self.is_square_free(p):
                return False
        return True

    def is_square_free(self, p):
        if not self.on_board(p):
            return False
        if not self.point_empty(p):
            return False
        return True

    @classmethod
    def on_board(self, point):
        valid_points = range(0, self.SIDE_LENGTH)
        return point.x in valid_points and point.y in valid_points
    
    def point_empty(self, p):
        val = self.check(p)
        logging.debug(val)
        return val == self.EMPTY
    
    def check(self, point):
        return self.board[point.y][point.x]
        
    def assign(self, point, value):
        if self.on_board(point):
            logging.debug("assigning %s: %s", point, value)
            self.board[point.y][point.x] = value
        else:
            logging.debug("%s off board not assigning", point)
            
    def __str__(self):
        return str(self.board)
    
    def copy(self):
        b = Board()
        b.board = self.board.copy()
        return b

    def display(self):
        vis = BoardView()
        vis.display(self.board)
        
def gen_pieces():
    pieces = {
        'p1': Piece([Point(0, 0)]),
        'p2': Piece([Point(0, 0), Point(0, 1)]),
        'p3': Piece([Point(0, 0), Point(0, 1), Point(0, 2)]),
        'p4': Piece([Point(0, 0), Point(0, 1), Point(1, 1)]),
        'p5': Piece([Point(0, 0), Point(1, 0), Point(0, 1), Point(1, 1)]),
        'p6': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(3, 0)]),
        'p7': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1)]),
        'p8': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 0)]),
        'p9': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1)]),
        'p10': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(3, 0), Point(4, 0)]),
        'p11': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(3, 0), Point(3, 1)]),
        'p12': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(2, 1), Point(3, 1)]),
        'p13': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(1, 1), Point(2, 1)]),
        'p14': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(0, 1), Point(2, 1)]),
        'p15': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(3, 0), Point(2, 1)]),
        'p16': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(1, 1), Point(1, 2)]),
        'p17': Piece([Point(0, 0), Point(1, 0), Point(2, 0), Point(2, 1), Point(2, 2)]),
        'p18': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1), Point(2, 2)]),
        'p19': Piece([Point(0, 0), Point(0, 1), Point(1, 1), Point(2, 1), Point(2, 2)]),
        'p20': Piece([Point(0, 0), Point(1, 0), Point(1, 1), Point(2, 1), Point(1, 2)]),
        'p21': Piece([Point(0, 1), Point(1, 1), Point(1, 2), Point(1, 0), Point(2, 1)])
    }
    
    return pieces

In [ ]:
def indent(obj, level):
    string = str(obj)
    join_str = "\n" + "\t" * level
    return "\t" * level + join_str.join(string.split("\n"))

In [ ]:
"""Valid Moves store ."""

class ValidMoves:
    
    def __init__(self):
        self.valid_moves = {}

    def add(self, move):
        try:
            corner_moves = self.valid_moves[move.corner]
            try:
                corner_moves[move.piece_id].append(move)
            except KeyError:
                corner_moves[move.piece_id] = [move]
        except KeyError:
            self.valid_moves[move.corner] = {}
            corner_moves = self.valid_moves[move.corner]
            corner_moves[move.piece_id] = [move]
    
    def remove(self, move):
        self.valid_moves[move.corner][move.piece_id].remove(move)
        
        if self.valid_moves[move.corner][move.piece_id] == []:
            del self.valid_moves[move.corner][move.piece_id]
    
        if self.valid_moves[move.corner] == {}:
            del self.valid_moves[move.corner]
        
    def get_all(self):
        gens = [self.get_corner_moves(c) for c in self.get_corners()]
        for m in itertools.chain(*gens):
            yield m

    def get_corner_moves(self, corner):
        for piece_id in self.valid_moves[corner]:
            for m in self.valid_moves[corner][piece_id]:
                yield m

    def get_corners(self):
        return self.valid_moves.keys()
    
    def get_corner_piece_moves(self, corner, piece_id):
        return filter(lambda m: m.piece_id == piece_id, self.get_corner_moves(corner))
            
    def get_piece_moves(self, piece_id):
        return filter(lambda m: m.piece_id == piece_id, self.get_valid_moves())

    def __len__(self):
        length = 0
        for val in self.valid_moves.values():
            for l in val.values():
                length += len(l)
        return length

In [ ]:
"""Valid Moves store ."""

class ValidMoves:
    
    def __init__(self):
        self.valid_moves = []

    def add(self, move):
        self.valid_moves.append(move)
    
    def remove(self, move):
        self.valid_moves.remove(move)
        
    def get_all(self):
        return self.valid_moves.copy()

    def get_corner_moves(self, corner):
        return filter(lambda x: x.corner == corner, self.get_all())
    
    def get_corner_piece_moves(self, corner, piece_id):
        return filter(lambda m: m.piece_id == piece_id, self.get_corner_moves(corner))
            
    def get_piece_moves(self, piece_id):
        return filter(lambda m: m.piece_id == piece_id, self.get_valid_moves())
    
    def get_corners(self):
        return list({m.corner for m in self.valid_moves})

    def __len__(self):
        return len(self.valid_moves)

In [ ]:
"""
This class defines a Player as a collection of.

Pieces, valid moves, and invalid squares.
"""

class Player:

    def __init__(self, player_id):
        self.player_id = player_id
        self.pieces = gen_pieces()
        self.valid_moves = ValidMoves()
        self.invalid_points = []
        
    def update(self, move):
        logging.info("Updating player %s", self.player_id)
        
        if self.player_id == move.player_id:

            logging.info("Deleting piece id: %s", move.piece_id)
            del self.pieces[move.piece_id]
            
            logging.info("Adding border points")
            self.add_border_points(move)
            
        logging.info("Clearing moves")
        self.clear_moves(move)
        
       

    def add_border_points(self, move):
        invalid_points = move.orientation.get_border_points()
        invalid_points = list(filter(Board.on_board, invalid_points))
        self.invalid_points += invalid_points

    def clear_moves(self, move):
        
        invalid_points = move.get_footprint()
        
        if self.player_id == move.player_id:
            invalid_points += move.orientation.get_border_points()
            
        valid_moves = list(self.valid_moves.get_all())

        for m in valid_moves:
            #if m == test_move:
            #    logger.setLevel(logging.DEBUG)
                
            if not self.is_move_valid(m):
                self.valid_moves.remove(m)
                continue
                
            if self.overlap(m, move):
                self.valid_moves.remove(m)
                
            #logger.setLevel(logging.INFO)

    def overlap(self, m1, m2):
        for p in m1.orientation.points:
            if p in m2.orientation.points:
                return True
        return False
            
    def is_move_valid(self, move):
        try:
            self.validate_move(move)
            return True
        except RuntimeError as err:
            logging.debug(err)
            return False

    def validate_move(self, move):
        if not self.has_piece(move.piece_id):
            raise RuntimeError(f"Move {move} invalid. Already played {move.piece_id}")

        for p in move.orientation.points:
            if p in self.invalid_points:
                raise RuntimeError(f"Move {move} invalid. Includes invalid point {p}")

    def has_piece(self, piece_id):
        return piece_id in self.pieces

    def get_corners(self):
        return self.valid_moves.get_corners()

    def add_move(self, move):
        self.valid_moves.add(move)

    def has_moves(self):
        return len(self.valid_moves) > 0

    def get_valid_moves(self):
        return self.valid_moves.get_all()

In [ ]:
"""A game of Blokus."""


class Game:

    def __init__(self):
        self.board = Board()
        self.player_pointer = 0
        self.players = [Player(i) for i in range(4)]
        self.set_starting_moves()
        self.move_history = []
    
    def set_starting_moves(self):
        corners = [Corner(Point(-1, -1), Point(0, 0)),
                   Corner(Point(-1, 20), Point(0, 19)),
                   Corner(Point(20, 20), Point(19, 19)),
                   Corner(Point(20, -1), Point(19, 0))]

        for i, c in enumerate(corners):
            self.add_corner_moves(c, i)

    def make_move(self, move):
        logging.info("Making move: %s", move)
        try:
            self.validate_move(move)
            self.update_state(move)
            self.move_history.append(move)
        except RuntimeError as err:
            logging.exception(err)

    def update_state(self, move):
        self.board.update(move)
        
        for player in self.players:
            player.update(move)
            
        for corner in move.orientation.get_corners():
            self.add_corner_moves(corner, move.player_id)
 
    def add_corner_moves(self, corner, player_id):
        player = self.players[player_id]
        rotation = corner.get_rotation()
        
        if not Board.on_board(corner.p2):
            logging.info("Corner %s is off board", corner)
            return
        
        logging.info("Adding corner %s moves", corner)
        for piece_id, piece in player.pieces.items():
            
            for orientation in piece.orientations:
                logging.debug("Examining orientation: %s", orientation)
                
                new_o = Orientation([corner.p2 + rotation(p) for p in orientation.points])
                logging.debug("New orientation: %s", new_o)
                m = Move(new_o, player_id, piece_id, corner)

                if self.is_move_valid(m):
                    logging.debug("Adding Move: %s", indent(m, 1))
                    player.add_move(m)

    def is_move_valid(self, move):
        try:
            self.validate_move(move)
            return True
        except RuntimeError as err:
            logging.debug(err)
            return False
    
    def validate_move(self, move):
        
        if not self.board.are_squares_free(move.orientation):
            raise RuntimeError(f"Move: {move} squares are not free")

        self.players[move.player_id].validate_move(move)
    
    def get_players_moves(self, player_id):
        return self.players[player_id].get_valid_moves()
    
    def display(self, player_id=None, bad_move=None):
        board = self.board
        
        if player_id is not None:
            board = self.fill_player_pov(board, player_id)
            
        if bad_move is not None:
            board = self.fill_bad_move(board, bad_move)

        board.display()

    def fill_player_pov(self, board, player_id):
        board = board.copy()
        
        adjacent_fill = 4
        corner_fill = 5
        
        for p in self.players[player_id].invalid_points:
            board.assign(p, adjacent_fill)
            
        for corner in self.players[player_id].get_corners():
            board.assign(corner.p2, corner_fill)
        return board

    def fill_bad_move(self, board, move):
        bad_fill = 6
        
        board = board.copy()

        for p in move.orientation.points:
            board.assign(p, bad_fill)
        return board

## Visualization Code

In [ ]:
class BoardView:

    def __init__(self):
        pass
    
    def display(self, board):
        n = 20
        my_dpi = 20
        canvasSize = (n,n)
        fig,ax = self.createCanvas(canvasSize[0],canvasSize[1],my_dpi)
        color = "black"

        for row in range(n):
            for col in range(n):
                color = self.get_color(board[row][col])
                rect = self.createRectangle((1,1),(col,row),canvasSize,color)
                ax.add_patch(rect)

        plt.show()

    def createCanvas(self, width,height,my_dpi):
        fig,ax = plt.subplots(1)
        fig.dpi= my_dpi
        fig.set_size_inches(width,height)
        ax.set_xticks([])
        ax.set_yticks([])
        return fig,ax

    def createRectangle(self, wh, xy, cwch, color="black"):
        """
        All units are in inches. 

        Parameters
        ==========
        wh: (width,height) 
        xy: (x,y)
        cwch: (canvasWidth,canvasHeight)
        """
        return patches.Rectangle((xy[0]/cwch[0],xy[1]/cwch[1]),wh[0]/cwch[0],wh[1]/cwch[1],color=color)

    def get_color(self, val):
        mapping = {
            -1: "#DCDCDC",
            0: "yellow",
            1: "red",
            2: "green",
            3: "blue",
            4: "orange",
            5: "purple",
            6: "black"
        }
        return mapping[val]

## Random Agent

In [ ]:
def get_random_move(board, player_id):
    if board.players[player_id].has_moves():        
        moves = list(board.get_players_moves(player_id))
        rand_int = random.randint(0, len(moves))
        return moves[rand_int]
    else:
        raise StopIteration
        
def get_mistake(board, player_id):
    if board.players[player_id].has_moves():        
        moves = list(board.get_players_moves(player_id))
        for m in moves:
            if not board.is_move_valid(m):
                return m
    else:
        return None

def replay_n_moves(moves, n):
    b = Board()
    for move in moves[:n]:
        board.make_move(move)
    return board

def replay_game(moves):
    return replay_n_moves(moves, len(moves))
    
def play_bad_move(moves):
    logger.setLevel(logging.WARNING)
    board = replay_game(moves[:-1])
    board.show_player_view(0)
    logger.setLevel(logging.DEBUG)
    board.make_move(moves[-1])

def play_random_moves():
    player_id = 0
    board = Board()
    while board.players[player_id].has_moves():
        m = get_random_move(board, player_id)
        board.make_move(m)
    return board

def get_random_move(board, player_id):
    if board.players[player_id].has_moves():        
        moves = list(board.get_players_moves(player_id))
        rand_int = random.randint(0, len(moves))
        return moves[rand_int]
    else:
        raise StopIteration

def look_for_mistakes():
    try:
        
        player_id = 1
        g = Game()
        while g.players[player_id].has_moves():
            m = get_random_move(g, player_id)
            g.make_move(m)
            g.display(player_id)
            for m in g.get_players_moves(player_id):
                if not g.is_move_valid(m):
                    return g.move_history
        print("Success")
    except Exception:
        return g.move_history

## Why player 1 no moves ???

In [ ]:
player_id = 1
g = Game()


## Status
Refactored but a bunch of cornsers get droped for no reason near the end

#### Questions:

why does it allow invalid moves in the below loop?

Theres an average branching factor of around 25
so call it 75 is that doable?





Number of chess game one guy says with 40 avg moves avg 30 moves per game
~ 10 ** 120 

James Hardy says (10 ** 50) ** 50 

Number of blockus game???

In [ ]:
f"{len(str(75 ** (20 * 4)))}"